LangChain의 출력파서(Output Parser)는 언어모델(LLM)의 출력을 더 유용하고 구조화된 형태로 변환하는 중요한 컴포넌트
### 출력파서의 역할
- LLM의 출력을 받아 더 적합한 형식으로 변환
- 구조화된 데이터 생성에 매우 유용
- LangChain 프레임워크에서 다양한 종류의 출력 데이터를 파싱, 처리

### 출력파서의 이점
- **구조화**: LLM의 자유 형식 텍스트 출력 -> 구조화된 데이터로 변환
- **일관성**: 출력 형식 일관되게 유지하여 후속 처리 용이
- **유연성**: 다양한 출력형식(JSON, 리스트, 딕셔너리 등)으로 변환 가능

### PydanticOutputParser
언어 모델의 출력을 더 구조화된 정보로 변환하는데 도움. **사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공**
- **get_format_instructions()**: 언어 모델이 출력해야 할 정보의 형식 정의.
- **parse():** 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환. Pydantic과 같은 도구를 사용하여, 입력된 문자열의 사전 정의된 스키마에 따라 검증, 변환

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith('OutputParser')

LangSmith 추적을 시작합니다.
[프로젝트명]
OutputParser


In [3]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [4]:
from itertools import chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_teddynote.messages import stream_response
prompt = PromptTemplate.from_template("다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}")

llm = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature= 0)

chain = prompt | llm
answer = chain.stream({"email_conversation": email_conversation})
output = stream_response(answer, return_output = True)

중요 내용 요약:

- 발신자: 김철수 (바이크코퍼레이션 상무)
- 수신자: 이은채 (Teddy International)
- 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- 요청 사항: ZENESIS 모델에 대한 상세 브로슈어 (기술 사양, 배터리 성능, 디자인 정보)
- 미팅 제안: 다음 주 화요일(1월 15일) 오전 10시, 귀사 사무실에서 만남 제안
- 목적: 협력 가능성 논의 및 유통 전략, 마케팅 계획 구체화

In [10]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [11]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"title": "Person", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c", "type": "string"}, "email": {"title": "Email", "description": "\uba54\uc77c\uc744 \ubcf4\ub0b8 \uc0ac\ub78c\uc758 \uc774\uba54\uc77c \uc8fc\uc18c", "type": "string"}, "subject": {"title": "Subject", "description": "\uba54\uc77c \uc81c\ubaa9", "type": "string"}, "summary": {"title": "Summary", "description": "\uba54\uc77c \ubcf8\ubb38\uc744 \uc694\uc57d\ud55c \ud14d\uc2a4\ud2b8", "type": "string"}, "date": {"title": "Date", "description"

1.**question**: 유저의 질문<br>
2.**email_conversation**: 이메일 본문의 내용 입력<br>
3.**format**: 형식을 지정합니다.<br>

In [9]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN. 

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [13]:
chain = prompt | llm
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무가 이은채 대리님에게 ZENESIS 자전거의 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다.",
  "date": "2024-01-15"
}
```

In [14]:
structured_output = parser.parse(output)

In [15]:
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='바이크코퍼레이션의 김철수 상무가 이은채 대리님에게 ZENESIS 자전거의 브로슈어 요청과 협력 논의를 위한 미팅 제안을 보냈습니다.' date='2024-01-15'
